In [1]:
import pandas as pd
import panel as pn
import numpy as np
import plotly.graph_objs as go
from scipy.stats import nbinom, binom
from pandas import Series
pn.extension()

In [2]:
# Generador de datos aleatorios
np.random.seed(42)
num_data_points = 200
tiempo = np.arange(num_data_points)

In [3]:
def calcular_crecimiento(tasa_nacimiento, tasa_muerte, celulas_iniciales):
    N = [celulas_iniciales]
    media_nbinom = tasa_nacimiento / tasa_muerte
    varianza_nbinom = media_nbinom / tasa_muerte
    for i in range(1, num_data_points):
        nacimiento = nbinom.rvs(n=varianza_nbinom, p=varianza_nbinom / (varianza_nbinom + media_nbinom), size=1)[0]        
        muerte = binom.rvs(n=N[-1], p=tasa_muerte, size=1)[0]
        N.append(N[-1] + nacimiento - muerte)
    return N


In [4]:
# Actualización de la grafica
def update_growth_curve(event):
    figure = go.Figure()
    j = rates()
    for i in range(4):
        tasa_nacimiento = j.values[i,0] /10
        tasa_muerte = j.values[i,1]/10
        celulas_iniciales = initial_cells_inputs[i].value
        datos_crecimiento = calcular_crecimiento(tasa_nacimiento, tasa_muerte, celulas_iniciales)
        figure.add_trace(go.Scatter(x=tiempo, y=datos_crecimiento, mode='lines', name=nombres[i]))

    figure.update_layout(
        title='Simulación del Crecimiento Celular (ONCOSIMULR)',
        xaxis_title='Tiempo',
        yaxis_title='Número de Células',
        legend=dict(orientation='h'),
    )
    growth_curve.object = figure
    
    j.insert(loc =0, column ='Genotipo', value = ['WT','BTC','Mph','MTC'])
    j = j.set_index('Genotipo')
    df_widget.value = j



In [5]:
#Widgets

#initial_cells_inputs = [pn.widgets.IntInput(name=f'Células Iniciales Curva {i+1}', value=100, step=10) for i in range(4)]

initial_cells_inputs = [pn.widgets.IntInput(name='CELULAS NATIVAS (NC)', start = 0, value = 100, step=10),
                       pn.widgets.IntInput(name='MACROFAGOS (Mph)', start = 0, value = 0, step=10),
                       pn.widgets.IntInput(name='CELULAS TUMORALES BENIGNAS (BTC)',start = 0,  value = 0, step=10),
                       pn.widgets.IntInput(name='CELULAS TUMORALES MÓTILES (MTC)', start = 0, value = 0, step=10)]

#caracteristicas
modify_character = [pn.widgets.FloatSlider(name="CG", start=0, end=10, step=0.1, value=2),
                    pn.widgets.FloatSlider(name='BG', start=0, end=10, step=0.1, value=5),
                    pn.widgets.FloatSlider(name='CS', start=0, end=10, step=0.1, value=1),
                    pn.widgets.FloatSlider(name='CM,Mph', start=0, end=10, step=0.01, value=0.8),
                    pn.widgets.FloatSlider(name='CM,MTC', start=0, end=10, step=0.01, value=1),
                    pn.widgets.FloatSlider(name='BR', start=0, end=10, step=0.1, value=1),
                    pn.widgets.FloatSlider(name='CD', start=0, end=10, step=0.1, value=9)]
modify_character[0].value

2

In [6]:
#Tabla
def table(birt, death):
    df = pd.DataFrame({'Genotipo': ['WT','BTC','Mph','MTC'], 'Birth': birt, 'Death': death}, index=[1, 2, 3,4])
    df= df.set_index('Genotipo')
    return df
    


In [7]:
#Creación de la tabla de nacimientos y muertes
def rates():
    poblacion_total = initial_cells_inputs[0].value + initial_cells_inputs[1].value + initial_cells_inputs[2].value + initial_cells_inputs[3].value;
    fnc = initial_cells_inputs[0].value/poblacion_total
    fmph = initial_cells_inputs[1].value/poblacion_total
    fbtc = initial_cells_inputs[2].value/poblacion_total
    tasas_de_nacimiento = [modify_character[1].value*(fnc + fmph),
                       modify_character[5].value + (modify_character[1].value*(fnc + fmph)),
                      modify_character[1].value*(fnc + fmph),
                      modify_character[5].value + (modify_character[1].value*(fnc + fmph))]
    tasas_de_muerte = [modify_character[0].value + (modify_character[2].value*(fnc + fbtc)),
                  (modify_character[2].value*(fnc + fbtc) + (modify_character[6].value*fmph)),
                  modify_character[0].value + modify_character[3].value,
                  modify_character[4].value + (modify_character[6].value * fmph)]
    D = pd.DataFrame({'Birth':tasas_de_nacimiento, 'Death':tasas_de_muerte})   
    return D

In [8]:
# Crear la gráfica inicial
figure = go.Figure()
nombres = ['NC', 'Mph', 'BTC', 'MTC']

j = rates()

for i in range(4):
    tasa_nacimiento = j.values[i,0] / 10
    tasa_muerte = j.values[i,1] / 10
    celulas_iniciales = initial_cells_inputs[i].value
    datos_crecimiento = calcular_crecimiento(tasa_nacimiento, tasa_muerte, celulas_iniciales)
    figure.add_trace(go.Scatter(x=tiempo, y=datos_crecimiento, mode='lines', name=nombres[i]))
    

figure.update_layout(
    title='Simulación del Crecimiento Celular (ONCOSIMULR)',
    xaxis_title='Tiempo',
    yaxis_title='Número de Células',
    legend=dict(orientation='h'),
)
growth_curve = pn.pane.Plotly(figure)
j.insert(loc =0, column ='Genotipo', value = ['WT','BTC','Mph','MTC'])
j = j.set_index('Genotipo')
df_widget = pn.widgets.DataFrame(j, name='DataFrame')

In [9]:
# Conectar los widgets con la función de actualización
for i in range(4):
    initial_cells_inputs[i].param.watch(update_growth_curve, 'value')
for j in range(7):
    modify_character[j].param.watch(update_growth_curve, 'value')

In [10]:
# Sidebar
def get_sidebar():
    sidebar = pn.Column(
        pn.pane.Markdown('## Población Inicial'),
        *[initial_cells_inputs[i] for i in range(4)],
        pn.pane.Markdown('## Caracteristicas de la población'),
        pn.pane.Markdown('Costo de producir el factor de crecimiento: '),
        *[modify_character[0]],
        pn.pane.Markdown('Beneficio de recibir el factor de crecimiento: '),
        *[modify_character[1]],
        pn.pane.Markdown('Costo de compartir los espacios: '),
        *[modify_character[2]],
        pn.pane.Markdown('Costo de la capacidad de moverse: '),
        *[modify_character[3]],
        pn.pane.Markdown('Costo de la capacidad de moverse para MTC: '),
        *[modify_character[4]],
        pn.pane.Markdown('Beneficio de reproducirse: '),
        *[modify_character[5]],
        pn.pane.Markdown('Costo de ser destruido por MACROFAGOS: '),
        *[modify_character[6]],
    )
    return sidebar

In [11]:
template = pn.template.FastListTemplate(title='Software de Simulación - Evolución Celular en un Cancer de Mama', sidebar=get_sidebar())
template.main.append(growth_curve)
template.main.append(pn.pane.Markdown('###Tasas de nacimiento y muerte.'))
template.main.append(df_widget)
template.show()

Launching server at http://localhost:40028
